In [2]:
cd drive/MyDrive/네카라쿠배/딥러닝/프로젝트

/content/drive/MyDrive/네카라쿠배/딥러닝/프로젝트


In [3]:
ls

baseline.ipynb       open/      resnet50_v2.pt     resnet50_v3.pt
baseline_result.csv  open_224/  resnet50_v3.csv    resnet50_ver1_sj.ipynb
checkpoiont_128.pt   open_448/  resnet50_v3.ipynb  resnet50_ver2_sj.ipynb


In [4]:
import torch
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
from torch import optim
from torch import nn

from torch.utils.data import Dataset
from torchvision.transforms import ToTensor
from torchvision import transforms

import random
from glob import glob
import pandas as pd
import numpy as np
from PIL import Image

In [62]:
from torchvision.transforms.transforms import Resize
from glob import glob

import pandas as pd
import numpy as np
from PIL import Image
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor
from torchvision import transforms

def extract_day(images):
    day = int(images.split('.')[-2][-2:])
    return day

def make_day_array(images):
    day_array = np.array([extract_day(x) for x in images])
    return day_array

def make_combination(length, species, data_frame, direct_name):
    before_file_path = []
    after_file_path = []
    time_delta = []

    for i in range(length):
        
        # 하위 폴더 중에서 랜덤하게 선택을 한다.
        direct = random.randrange(0,len(direct_name))
        # 위에서 결정된 폴더를 선택한다. 
        temp = data_frame[data_frame['version'] == direct_name[direct]]
    
        # 밑은 기존의 코드와 동일합니다.
        sample = temp[temp['species'] == species].sample(2)
        after = sample[sample['day'] == max(sample['day'])].reset_index(drop=True)
        before = sample[sample['day'] == min(sample['day'])].reset_index(drop=True)

        before_file_path.append(before.iloc[0]['file_name'])
        after_file_path.append(after.iloc[0]['file_name'])
        delta = int(after.iloc[0]['day'] - before.iloc[0]['day'])
        time_delta.append(delta)

    combination_df = pd.DataFrame({
        'before_file_path': before_file_path,
        'after_file_path': after_file_path,
        'time_delta': time_delta,
    })

    combination_df['species'] = species

    return combination_df

class TrainDataset(Dataset):
    def __init__(self, combination_df, is_test=None):
        self.combination_df = combination_df
        self.transform = transforms.Compose([
            transforms.Resize(256),
            transforms.RandomCrop(224),                                    
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomVerticalFlip(p=0.5),
            transforms.RandomAffine((-20, 20)),
            transforms.RandomRotation(90),                                 
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        self.is_test = is_test

    def __getitem__(self, idx):
        before_image = Image.open(self.combination_df.iloc[idx]['before_file_path'])
        after_image = Image.open(self.combination_df.iloc[idx]['after_file_path'])

        before_image = self.transform(before_image)
        after_image = self.transform(after_image)
        if self.is_test:
            return before_image, after_image
        time_delta = self.combination_df.iloc[idx]['time_delta']
        return before_image, after_image, time_delta

    def __len__(self):
        return len(self.combination_df)

class TestDataset(Dataset):
    def __init__(self, combination_df, is_test=None):
        self.combination_df = combination_df
        self.transform = transforms.Compose([                                                             
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        self.is_test = is_test

    def __getitem__(self, idx):
        before_image = Image.open(self.combination_df.iloc[idx]['before_file_path'])
        after_image = Image.open(self.combination_df.iloc[idx]['after_file_path'])

        before_image = self.transform(before_image)
        after_image = self.transform(after_image)
        if self.is_test:
            return before_image, after_image
        time_delta = self.combination_df.iloc[idx]['time_delta']
        return before_image, after_image, time_delta

    def __len__(self):
        return len(self.combination_df)

In [63]:
import torch
from torch import nn
from torchvision.models import resnet50


class CompareCNN(nn.Module):

    def __init__(self):
        super(CompareCNN, self).__init__()
        self.resnet = resnet50(pretrained=True)
        self.fc_layer = nn.Linear(1000, 1)
        
    def forward(self, input):
        x = self.resnet(input)
        output = self.fc_layer(x)
        return output

class CompareNet(nn.Module):

    def __init__(self):
        super(CompareNet, self).__init__()
        self.before_net = CompareCNN()
        self.after_net = CompareCNN()

        before_model = resnet50(pretrained=True)                           # 미세조정 위해서 추가
        before_model.fc_layer = nn.Linear(1000,1)
        before_modules = list(before_model.children())[:-1]                # 미세조정 위해서 추가
        self.before_feature_extract_model = nn.Sequential(*before_modules) # 미세조정 위해서 추가

        after_model = resnet50(pretrained=True)                          # 미세조정 위해서 추가
        after_model.fc_layer = nn.Linear(1000,1)
        after_modules = list(after_model.children())[:-1]                # 미세조정 위해서 추가
        self.after_feature_extract_model = nn.Sequential(*after_modules) # 미세조정 위해서 추가


    def forward(self, before_input, after_input):
        before = self.before_net(before_input)
        after = self.after_net(after_input)
        delta = before - after
        return delta

In [64]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [66]:
def seed_everything(seed): # seed 고정
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)


seed_everything(2048)
# 학습 데이터가 있는 폴더 위치
root_path = './open_224/train_dataset/'

# BC 폴더와 LT 폴더에 있는 하위 폴더를 저장한다.
bc_direct = glob(root_path + '/BC/*')
bc_direct_name = [x[-5:] for x in bc_direct]
lt_direct = glob(root_path + '/LT/*')
lt_direct_name = [x[-5:] for x in lt_direct]

# 하위 폴더에 있는 이미지들을 하위 폴더 이름과 매칭시켜서 저장한다.
bc_images = {key : glob(name + '/*.png') for key,name in zip(bc_direct_name, bc_direct)}
lt_images = {key : glob(name + '/*.png') for key,name in zip(lt_direct_name, lt_direct)}

# 하위 폴더에 있는 이미지들에서 날짜 정보만 따로 저장한다.
bc_dayes = {key : make_day_array(bc_images[key]) for key in bc_direct_name}
lt_dayes = {key : make_day_array(lt_images[key]) for key in lt_direct_name}

bc_dfs = []
for i in bc_direct_name:
    bc_df = pd.DataFrame({
        'file_name':bc_images[i],
        'day':bc_dayes[i],
        'species':'bc',
        'version':i
    })
    bc_dfs.append(bc_df)
    
lt_dfs = []
for i in lt_direct_name:
    lt_df = pd.DataFrame({
        'file_name':lt_images[i],
        'day':lt_dayes[i],
        'species':'lt',
        'version':i
    })
    lt_dfs.append(lt_df)

bc_dataframe = pd.concat(bc_dfs).reset_index(drop=True)
lt_dataframe = pd.concat(lt_dfs).reset_index(drop=True)
total_dataframe = pd.concat([bc_dataframe, lt_dataframe]).reset_index(drop=True)

bc_combination = make_combination(5000, 'bc', total_dataframe, bc_direct_name)
lt_combination = make_combination(5000, 'lt', total_dataframe, lt_direct_name)

bc_train = bc_combination.iloc[:4500]
bc_valid = bc_combination.iloc[4500:]

lt_train = lt_combination.iloc[:4500]
lt_valid = lt_combination.iloc[4500:]

train_set = pd.concat([bc_train, lt_train])
valid_set = pd.concat([bc_valid, lt_valid])



train_dataset = TrainDataset(train_set)
valid_dataset = TestDataset(valid_set)

In [67]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [68]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
lr = 1e-5
epochs = 10
batch_size = 32
valid_batch_size = 50

model = CompareNet().to(device)

optimizer = optim.Adam(model.parameters(), lr=lr)

train_data_loader = DataLoader(train_dataset,
                               batch_size=batch_size,
                               shuffle=True)

valid_data_loader = DataLoader(valid_dataset,
                               batch_size=valid_batch_size)

In [ ]:
model

## 미세조정 시도

In [70]:
before_model_ft = model.before_feature_extract_model
ct = 0
for child in before_model_ft.children():
    ct += 1
    if ct < 8:
        for param in child.parameters():
            param.requires_grad = False

after_model_ft = model.after_feature_extract_model
ct = 0
for child in after_model_ft.children():
    ct += 1
    if ct < 8:
        for param in child.parameters():
            param.requires_grad = False

- 레이어 갯수 잘 확인

In [71]:
tmp = model.before_feature_extract_model

In [72]:
i = 1
for child in tmp.children():
    print(i)
    print(child)
    i += 1

1
Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
2
BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
3
ReLU(inplace=True)
4
MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
5
Sequential(
  (0): Bottleneck(
    (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (downsample): Sequential(
      (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affi

In [73]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
for epoch in tqdm(range(epochs)):
    for step, (before_image, after_image, time_delta) in tqdm(enumerate(train_data_loader)):
        before_image = before_image.to(device)
        after_image = after_image.to(device)
        time_delta = time_delta.to(device)

        optimizer.zero_grad()
        logit = model(before_image, after_image)
        train_loss = (torch.sum(torch.abs(logit.squeeze(1).float() - time_delta.float())) /
                      torch.LongTensor([batch_size]).squeeze(0).to(device))
        train_loss.backward()
        optimizer.step()

        if step % 15 == 0:
            print('\n=====================loss=======================')
            print(f'\n=====================EPOCH: {epoch}=======================')
            print(f'\n=====================step: {step}=======================')
            print('MAE_loss : ', train_loss.detach().cpu().numpy())

    valid_losses = []
    with torch.no_grad():
        for valid_before, valid_after, time_delta in tqdm(valid_data_loader):
            valid_before = valid_before.to(device)
            valid_after = valid_after.to(device)
            valid_time_delta = time_delta.to(device)


            logit = model(valid_before, valid_after)
            valid_loss = (torch.sum(torch.abs(logit.squeeze(1).float() - valid_time_delta.float())) /
                          torch.LongTensor([valid_batch_size]).squeeze(0).to(device))
            valid_losses.append(valid_loss.detach().cpu())


    print(f'VALIDATION_LOSS MAE : {sum(valid_losses)/len(valid_losses)}')
    checkpoint = {
        'model': model.state_dict(),

    }

    torch.save(checkpoint, 'resnet50_v3.pt')

  0%|          | 0/10 [00:00<?, ?it/s]

0it [00:00, ?it/s]


=====================loss=======================

=====================EPOCH: 0=======================

=====================step: 0=======================
MAE_loss :  11.439497

=====================loss=======================

=====================EPOCH: 0=======================

=====================step: 15=======================
MAE_loss :  6.492715

=====================loss=======================

=====================EPOCH: 0=======================

=====================step: 30=======================
MAE_loss :  8.913658

=====================loss=======================

=====================EPOCH: 0=======================

=====================step: 45=======================
MAE_loss :  5.4048166

=====================loss=======================

=====================EPOCH: 0=======================

=====================step: 60=======================
MAE_loss :  3.8787313

=====================loss=======================

=====================EPOCH: 0=======================

  0%|          | 0/20 [00:00<?, ?it/s]

VALIDATION_LOSS MAE : 2.7893481254577637


...


=====================loss=======================

=====================EPOCH: 2=======================

=====================step: 0=======================
MAE_loss :  1.7002157

=====================loss=======================

=====================EPOCH: 2=======================

=====================step: 15=======================
MAE_loss :  1.8647479

=====================loss=======================

=====================EPOCH: 2=======================

=====================step: 30=======================
MAE_loss :  1.3340646

=====================loss=======================

=====================EPOCH: 2=======================

=====================step: 45=======================
MAE_loss :  2.7781532

=====================loss=======================

=====================EPOCH: 2=======================

=====================step: 60=======================
MAE_loss :  2.9697325

=====================loss=======================

=====================EPOCH: 2=====================

  0%|          | 0/20 [00:00<?, ?it/s]

VALIDATION_LOSS MAE : 2.535022020339966


0it [00:00, ?it/s]


=====================loss=======================

=====================EPOCH: 3=======================

=====================step: 0=======================
MAE_loss :  2.1397147

=====================loss=======================

=====================EPOCH: 3=======================

=====================step: 15=======================
MAE_loss :  2.053712

=====================loss=======================

=====================EPOCH: 3=======================

=====================step: 30=======================
MAE_loss :  1.4162827

=====================loss=======================

=====================EPOCH: 3=======================

=====================step: 45=======================
MAE_loss :  4.6501284

=====================loss=======================

=====================EPOCH: 3=======================

=====================step: 60=======================
MAE_loss :  2.661262

=====================loss=======================

=====================EPOCH: 3=======================

In [ ]:
test_set = pd.read_csv('./open_224/test_dataset/test_data.csv')
test_set['l_root'] = test_set['before_file_path'].map(lambda x: './open_224/test_dataset/' + x.split('_')[1] + '/' + x.split('_')[2])
test_set['r_root'] = test_set['after_file_path'].map(lambda x: './open_224/test_dataset/' + x.split('_')[1] + '/' + x.split('_')[2])
test_set['before_file_path'] = test_set['l_root'] + '/' + test_set['before_file_path'] + '.png'
test_set['after_file_path'] = test_set['r_root'] + '/' + test_set['after_file_path'] + '.png'

test_dataset = TestDataset(test_set, is_test=True)
test_data_loader = DataLoader(test_dataset,
                               batch_size=64)

In [20]:
test_value = []
with torch.no_grad():
    for test_before, test_after in tqdm(test_data_loader):
        test_before = test_before.to(device)
        test_after = test_after.to(device)
        logit = model(test_before, test_after)
        value = logit.squeeze(1).detach().cpu().float()
        
        test_value.extend(value)

  0%|          | 0/62 [00:00<?, ?it/s]

In [21]:
# submission 형식을 불러온다.
submission = pd.read_csv('./open_224/sample_submission.csv')

# 예측한 값들은 텐서 형태로 변환 시켜준다.
predict = torch.FloatTensor(test_value)

# 음수의 값을 갖는 모든 값들을 1 Day 차이가 발생하도록 바꿔줌
temp_predict = predict.numpy()
temp_predict[np.where(temp_predict<1)] = 1

# 모델의 예측 값을 저장함
submission['time_delta'] = temp_predict
submission.to_csv('resnet50_fine_tuning.csv', index=False)